In [1]:
from langchain_groq import ChatGroq

In [2]:
llm = ChatGroq(
    temperature=0, 
    groq_api_key='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', 
    model_name="llama-3.3-70b-versatile"
)
response = llm.invoke("The first person to land on moon was ...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to set foot on the moon.


In [3]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://careers.nike.com/software-engineer-iii/job/R-49238")
page_data = loader.load().pop().page_content
print(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.























Software Engineer III










































Skip to main content
Open Virtual Assistant










Home


Career Areas


Total Rewards


Life@Nike


Purpose










Language





Select a Language

  Deutsch  
  English  
  Español (España)  
  Español (América Latina)  
  Français  
  Italiano  
  Nederlands  
  Polski  
  Tiếng Việt  
  Türkçe  
  简体中文  
  繁體中文  
  한국어  
  日本語  








Careers


















Close Menu







Careers






Chat






                                Home
                            



                                Career Areas
                            



                                Total Rewards
                            



                                Life@Nike
                            



                                Purpose
                            










Jordan Careers







Converse Careers










Language











Menu



Return to Previous Menu



Select a Languag

In [4]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [5]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Software Engineer III',
 'experience': '8 years of practical application or 5 years with advanced education',
 'skills': ['Java development',
  'cloud computing platforms',
  'microservices architecture',
  'containerization',
  'infrastructure as code',
  'configuration management tools',
  'CI/CD build systems'],
 'description': 'The Enterprise Cloud team is responsible for Nike’s global cloud strategy and evolution to ensure we continue to meet the changing needs of the business. The ideal team member brings a growth mindset and demonstrated experience in building scalable cloud solutions.'}

In [6]:
type(json_res)

dict

In [7]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [8]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [9]:
job = json_res
#job['skills']

In [10]:
job

{'role': 'Software Engineer III',
 'experience': '8 years of practical application or 5 years with advanced education',
 'skills': ['Java development',
  'cloud computing platforms',
  'microservices architecture',
  'containerization',
  'infrastructure as code',
  'configuration management tools',
  'CI/CD build systems'],
 'description': 'The Enterprise Cloud team is responsible for Nike’s global cloud strategy and evolution to ensure we continue to meet the changing needs of the business. The ideal team member brings a growth mindset and demonstrated experience in building scalable cloud solutions.'}

In [11]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/java-portfolio'},
  {'links': 'https://example.com/android-portfolio'}],
 [{'links': 'https://example.com/xamarin-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/xamarin-portfolio'},
  {'links': 'https://example.com/android-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/ios-ar-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/full-stack-js-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/kotlin-backend-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}]]

In [12]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Prinkle Singharia. An ML/AI Engineer based in Boston, MA.
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of Prinkle 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Prinkle's portfolio: {link_list}
        Remember you are Prinkle, an ML/AI Engineer.
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Application for Software Engineer III Role at Nike

Dear Hiring Manager,

I am excited to apply for the Software Engineer III position at Nike, as advertised. With 8 years of experience in practical application and a strong background in machine learning and artificial intelligence, I am confident that I can bring significant value to the Enterprise Cloud team.

As an ML/AI Engineer, I have developed a strong foundation in Java development, which I believe is essential for building scalable cloud solutions. My experience with cloud computing platforms, microservices architecture, and containerization will enable me to make a seamless transition into this role. Additionally, I have worked with infrastructure as code, configuration management tools, and CI/CD build systems, which aligns with the requirements of this position.

I would like to showcase some of my relevant work, which demonstrates my capabilities in building scalable cloud solutions. Please find some examples of m